# ReDimNetNoMel Disable bad layers

In [25]:
import torch
print(torch.__version__)
from torchsummary import summary


import torchaudio
import torchaudio.transforms as T

import torch.nn as nn
import torch.nn.functional as F

import numpy as np


2.6.0+cu124


In [26]:
model_name='B0'
# train_type='ft_lm'
train_type='ptn'
dataset='vox2'

torch.hub.set_dir('/data/deep/redimnet/models')

original_model = torch.hub.load('IDRnD/ReDimNet', 'ReDimNet', 
                       model_name=model_name, 
                       train_type=train_type, 
                       dataset=dataset)

/data/deep/redimnet/models/IDRnD_ReDimNet_master
load_res : <All keys matched successfully>


Using cache found in /data/deep/redimnet/models/IDRnD_ReDimNet_master


*  ReDimNetWrap expects raw 16 kHz mono audio, exactly 32 000 samples

In [27]:
from torchinfo import summary
summary(original_model, input_size=(1, 32000))

Layer (type:depth-idx)                                       Output Shape              Param #
ReDimNetWrap                                                 [1, 192]                  --
├─MelBanks: 1-1                                              [1, 60, 134]              --
│    └─Sequential: 2-1                                       [1, 60, 134]              --
│    │    └─Identity: 3-1                                    [1, 32000]                --
│    │    └─PreEmphasis: 3-2                                 [1, 32000]                --
│    │    └─MelSpectrogram: 3-3                              [1, 60, 134]              --
├─ReDimNet: 1-2                                              [1, 600, 134]             --
│    └─Sequential: 2-2                                       [1, 600, 134]             --
│    │    └─Conv2d: 3-4                                      [1, 10, 60, 134]          100
│    │    └─LayerNorm: 3-5                                   [1, 10, 60, 134]          20
│   

we can see MelSpectrogram inside the model ; lets take it outside the model;


In [28]:
for name, module in original_model.named_children():
    print(name, "=>", module)

backbone => ReDimNet(
  (stem): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): LayerNorm(C=(10,), data_format=channels_first, eps=1e-06)
    (2): to1d()
  )
  (stage0): Sequential(
    (0): weigth1d(w=(1, 1, 1, 1),sequential=False)
    (1): to2d(f=60,c=10)
    (2): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
    (3): ConvBlock2d(
      (conv_block): ResBasicBlock(
        (conv1): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
        (conv1pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
        (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
        (conv2pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
        (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      

# create new model

In [29]:
########################################
# 2) Define a Model Class without MelBanks
########################################
import torch
import torch.nn as nn

class ReDimNetNoMel(nn.Module):
    """
    A wrapper around the original ReDimNetWrap that:
      - Excludes the 'spec' (MelBanks) module
      - Uses 'backbone', 'pool', 'bn', and 'linear'
    We expect a precomputed mel spectrogram as input with shape [B, 1, n_mels, time_frames].
    """
    def __init__(self, original_wrap):
        super().__init__()
        
        # Grab references to the submodules we want to keep
        self.backbone = original_wrap.backbone
        
        
        ## DESERT SEARCH WHAT HANGS ###
        # self.backbone.stage0[6] = nn.Identity()
        # self.backbone.stage1[8] = nn.Identity()
        # self.backbone.stage2[8] = nn.Identity()
        # self.backbone.stage3[9] = nn.Identity()
        # self.backbone.stage4[7] = nn.Identity()
        ### >>>> PASS
        
        # try: only close TransformerEncoderLayer
        # self.backbone.stage0[6].tcm[4] = nn.Identity()
        # self.backbone.stage1[8].tcm[4] = nn.Identity()
        # self.backbone.stage2[8].tcm[4] = nn.Identity()
        # self.backbone.stage3[9].tcm[4] = nn.Identity()
        # self.backbone.stage4[7].tcm[4] = nn.Identity()
        # >>>> NOT PASS
        
        # try: close only 1 ConvNeXtLikeBlock
        # self.backbone.stage0[6].tcm[0] = nn.Identity()
        # self.backbone.stage1[8].tcm[0] = nn.Identity()
        # self.backbone.stage2[8].tcm[0] = nn.Identity()
        # self.backbone.stage3[9].tcm[0] = nn.Identity()
        # self.backbone.stage4[7].tcm[0] = nn.Identity()
        # >>> NOT PASS
        
        ## try: all layers of ConvNeXtLikeBlock
        for stage_idx, block_idx in [(0, 6), (1, 8), (2, 8), (3, 9), (4, 7)]:
            for tcm_idx in range(4):  # tcm[0] to tcm[3] 
                # self.backbone.__getattr__(f'stage{stage_idx}')[block_idx].tcm[tcm_idx] = nn.Identity()            ## >>>> PASS
                # self.backbone.__getattr__(f'stage{stage_idx}')[block_idx].tcm[tcm_idx].act = nn.SiLU()            ## >>>> NOT PASS
                ## !!! all Conv1d
                self.backbone.__getattr__(f'stage{stage_idx}')[block_idx].tcm[tcm_idx].dwconvs[0] = nn.Identity()
                self.backbone.__getattr__(f'stage{stage_idx}')[block_idx].tcm[tcm_idx].pwconv1 = nn.Identity()
                ## >>>> PASS

        
        
        # Replace ASTP with RKNN-safe version:
        self.pool = original_wrap.pool
        self.bn = original_wrap.bn
        self.linear = original_wrap.linear

    def forward(self, x):
        # x: shape [B, 1, n_mels, time_frames]
        # (1) Pass through the backbone
        x = self.backbone(x)    # shape might become [B, channels, frames] or similar
        print("Backbone output shape:", x.shape)  # ADD THIS LINE
        # (2) Pooling
        x = self.pool(x)        # ASTP => shape likely [B, embedding_dim]
        # (3) BatchNorm
        x = self.bn(x)
        # (4) Final linear => 192-dim (if that's your embedding size)
        x = self.linear(x)
        return x


# Create an instance of our new model that skips the MelBanks front-end
model_no_mel = ReDimNetNoMel(original_model)



In [30]:
model_no_mel.eval()  # <- this line is critical!
dummy = torch.randn(1, 1, 60, 200)
model_no_mel(dummy)

Backbone output shape: torch.Size([1, 600, 200])


tensor([[  3.5160,  -0.4834,   0.7326,   0.7741,   2.0211,   0.5439,  -0.6739,
           0.4966,   0.9468,   6.7896,   4.7158,   0.8735,  -2.5348,  -5.9167,
          -4.2186,   3.4529,   0.5521,   2.9442,  -0.3972,  -5.5993,   3.1930,
           1.2385,  -1.2525,  -2.5227,   0.5463,  -0.2271,   2.9769,   1.3343,
          -3.9380,  -7.8455,  -1.0612,  -0.8553,  -4.4971,   1.7912,   4.2034,
           4.0984,  -0.2862,  -1.0009,  -0.5584,   0.9492,   0.8864,   2.4532,
          -3.5641,  -2.7309,   4.0435,   0.6010,  -1.0650,  -2.0920,   3.8576,
           3.4443,  -1.8766,  -4.6215,  -2.9233,   2.6333,  -4.4315,  -1.9602,
           0.9660,  -0.5026,  -1.2816,  -0.7662,  -3.9599,  -1.6898,   2.1993,
           3.8689,  -0.2117,   1.7614,   1.2466,   3.5478,  -2.2862,  -0.3704,
           1.4137,  -5.0253,  -6.2131,  -0.8037,   1.5271,  -2.5333,  -3.5015,
           3.8416,   1.0883,  -3.6144,  -4.2369,   3.4160,   1.8182,   6.4865,
           1.0032,  -0.5237,   0.2366,  -2.3312,   0

## layres debug

problematic layer tree is:

```
TimeContextBlock1d
├── red_dim_conv (Sequential)
│   ├── Conv1d(600 → 60, kernel_size=1)
│   └── LayerNorm(C=60, data_format=channels_first)
├── tcm (Sequential)
│   ├── ConvNeXtLikeBlock (kernel=7)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=7, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   ├── ConvNeXtLikeBlock (kernel=19)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=19, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   ├── ConvNeXtLikeBlock (kernel=31)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=31, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   ├── ConvNeXtLikeBlock (kernel=59)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=59, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   └── TransformerEncoderLayer
│       ├── attention (MultiHeadAttention)
│       │   ├── k_proj: Linear(60 → 60)
│       │   ├── v_proj: Linear(60 → 60)
│       │   ├── q_proj: Linear(60 → 60)
│       │   └── out_proj: Linear(60 → 60)
│       ├── layer_norm: LayerNorm(60)
│       ├── feed_forward
│       │   ├── intermediate_dropout: Dropout(0.0)
│       │   ├── intermediate_dense: Linear(60 → 60)
│       │   ├── intermediate_act_fn: NewGELUActivation
│       │   ├── output_dense: Linear(60 → 60)
│       │   └── output_dropout: Dropout(0.0)
│       └── final_layer_norm: LayerNorm(60)
└── exp_dim_conv: Conv1d(60 → 600, kernel_size=1)
```

In [31]:
for name, module in model_no_mel.named_modules():
    if isinstance(module, nn.LayerNorm):
        print("❌ Still has LayerNorm at:", name)

❌ Still has LayerNorm at: backbone.stage0.6.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage0.6.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage1.8.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage1.8.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage2.8.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage2.8.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage3.9.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage3.9.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage4.7.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage4.7.tcm.4.final_layer_norm


In [32]:
print("stage0.6 =", model_no_mel.backbone.stage4[7].tcm[0]) 

stage0.6 = ConvNeXtLikeBlock(
  (dwconvs): ModuleList(
    (0): Identity()
  )
  (norm): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): GELU(approximate='none')
  (pwconv1): Identity()
)


## info

In [33]:
model_no_mel.eval()


ReDimNetNoMel(
  (backbone): ReDimNet(
    (stem): Sequential(
      (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): LayerNorm(C=(10,), data_format=channels_first, eps=1e-06)
      (2): to1d()
    )
    (stage0): Sequential(
      (0): weigth1d(w=(1, 1, 1, 1),sequential=False)
      (1): to2d(f=60,c=10)
      (2): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
      (3): ConvBlock2d(
        (conv_block): ResBasicBlock(
          (conv1): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
          (conv1pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
          (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
          (conv2pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
          (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [34]:
summary(model_no_mel, (1, 1, 60, 200))


Backbone output shape: torch.Size([1, 600, 200])


Layer (type:depth-idx)                                       Output Shape              Param #
ReDimNetNoMel                                                [1, 192]                  --
├─ReDimNet: 1-1                                              [1, 600, 200]             --
│    └─Sequential: 2-1                                       [1, 600, 200]             --
│    │    └─Conv2d: 3-1                                      [1, 10, 60, 200]          100
│    │    └─LayerNorm: 3-2                                   [1, 10, 60, 200]          20
│    │    └─to1d: 3-3                                        [1, 600, 200]             --
│    └─Sequential: 2-2                                       [1, 600, 200]             --
│    │    └─weigth1d: 3-4                                    [1, 600, 200]             (1)
│    │    └─to2d: 3-5                                        [1, 10, 60, 200]          --
│    │    └─Conv2d: 3-6                                      [1, 10, 60, 200]          110
│ 

## Utility Function for WAV -> MelSpectrogram

In [35]:

# ------------------------------------------------------------------
#  ReDimNet front-end settings (taken from the IDRnD repo defaults)
#    • 16 kHz audio
#    • pre-emphasis α = 0.97
#    • 25 ms window  (400 samples)
#    • 15 ms hop     (240 samples)  ➜ 134 frames for a 2-s clip
#    • 60 Mel bins, 20 Hz → 8 kHz
# ------------------------------------------------------------------
_PREEMPH  = 0.97
_SR       = 16_000
_N_FFT    = 512
_WIN_LEN  = 400
_HOP      = 240
_N_MELS   = 60
_F_MIN    = 20.0
_F_MAX    = 7600.0
_EPS      = 1e-6            # numerical stability


# Singleton MelSpectrogram so we build the kernel only once
_mel_layer = T.MelSpectrogram(
    sample_rate=_SR,
    n_fft=_N_FFT,
    win_length=_WIN_LEN,
    hop_length=_HOP,
    f_min=_F_MIN,
    f_max=_F_MAX,
    n_mels=_N_MELS,
    power=2.0,               # the original uses power-spec → log10 later
    center=True,
    pad_mode="reflect",
    window_fn=torch.hamming_window
)

def _pre_emphasis(wave: torch.Tensor, alpha: float = _PREEMPH) -> torch.Tensor:
    """y[n] = x[n] − α·x[n−1] (first sample unchanged)."""
    y = wave.clone()
    y[:, 1:] = y[:, 1:] - alpha * y[:, :-1]
    return y


def pad_or_crop_logmel(log_mel, target_frames=200):
    """
    Ensures log_mel is shaped [1, n_mels, target_frames] by:
    - Padding with zeros on the right if too short
    - Center-cropping if too long
    """
    B, M, T = log_mel.shape
    if T < target_frames:
        pad_amt = target_frames - T
        log_mel = F.pad(log_mel, (0, pad_amt))  # pad at end
        print(f"Padding log_mel from {T} to {target_frames} frames")
    elif T > target_frames:
        start = (T - target_frames) // 2
        log_mel = log_mel[:, :, start:start + target_frames]
        print(f"Cropping log_mel from {T} to {target_frames} frames")
    return log_mel


@torch.no_grad()
def waveform_to_logmel(wave: torch.Tensor) -> torch.Tensor:
    """
    Parameters
    ----------
    wave : Tensor [B', T] | [1, T]
        16-kHz mono waveform already trimmed / padded (32 000 samples for 2 s).

    Returns
    -------
    log_mel : Tensor [B', 1, 60, frames]
        Bit-exact front-end output expected by `model_no_mel`.
    """
    # Make sure we always have a batch dimension
    if wave.dim() == 1:      # (T,) → (1, T)
        wave = wave.unsqueeze(0)
    elif wave.dim() == 2 and wave.shape[0] > 1:
        raise ValueError("Input must be mono; got multi-channel tensor.")

    # pre-emphasis
    wave = _pre_emphasis(wave.float())

    # Mel power-spectrogram
    mel = _mel_layer(wave)
    mel = torch.log(mel + 1e-6)          # → [B, 60, frames]

    # log-scale (natural or log10 both work – log10 matches repo)
    mel = mel - mel.mean(dim=-1, keepdim=True)

    # pad/crop
    mel = pad_or_crop_logmel(mel, target_frames=200)  # Ensure 200 frames

    # add the dummy channel dim expected by Conv2d stem
    mel = mel.unsqueeze(1)                # → [B, 1, 60, frames]

    return mel

In [36]:
def example_inference(wav_path: str):
    # (a) Load audio
    waveform, sample_rate = torchaudio.load(wav_path)  # shape: [channels, time]
    # If stereo, select one channel, or average:
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
        
    # Resample if needed
    target_sample_rate=16000
    if sample_rate != target_sample_rate:
        resampler = T.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = resampler(waveform)

    # (b) Convert to log-mel
    log_mel = waveform_to_logmel(waveform)
    print('feeding logmel shape:', log_mel.shape)
    
    # (c) Forward pass
    with torch.no_grad():
        embedding = model_no_mel(log_mel)  # shape typically [1, 192] or so

    print("Embedding shape:", embedding.shape)
    #print("Embedding:", embedding)
    return embedding

In [37]:
# Compute similarity between two embeddings
def cosine_similarity(embedding1, embedding2):
    return F.cosine_similarity(embedding1, embedding2).item()

def cosine_similarity_numpys(emb1: np.ndarray, emb2: np.ndarray) -> float:
    """
    Compute cosine similarity between two vectors of shape (D,) or (1, D).
    """
    # If shape is (1, D), flatten to (D,)
    v1 = emb1.flatten()
    v2 = emb2.flatten()

    # dot product
    dot = np.dot(v1, v2)
    # norms
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)

    # Add a small epsilon in case of very small norms
    sim = dot / (norm1 * norm2 + 1e-8)
    return sim


In [38]:
embed1 = example_inference("testRob1.wav")
embed2 = example_inference("testRob2.wav")
embed3 = example_inference("testme1.wav")
embed4 = example_inference("testme2.wav")


Padding log_mel from 108 to 200 frames
feeding logmel shape: torch.Size([1, 1, 60, 200])
Backbone output shape: torch.Size([1, 600, 200])
Embedding shape: torch.Size([1, 192])
Padding log_mel from 99 to 200 frames
feeding logmel shape: torch.Size([1, 1, 60, 200])
Backbone output shape: torch.Size([1, 600, 200])
Embedding shape: torch.Size([1, 192])
Padding log_mel from 118 to 200 frames
feeding logmel shape: torch.Size([1, 1, 60, 200])
Backbone output shape: torch.Size([1, 600, 200])
Embedding shape: torch.Size([1, 192])
Cropping log_mel from 222 to 200 frames
feeding logmel shape: torch.Size([1, 1, 60, 200])
Backbone output shape: torch.Size([1, 600, 200])
Embedding shape: torch.Size([1, 192])


In [39]:
print(f"Similarity (robot to robot): {cosine_similarity_numpys(embed1, embed2)}")
print(f"Similarity (robot to me   ): {cosine_similarity_numpys(embed1, embed3)}")
print(f"Similarity (me 1 to me 2  ): {cosine_similarity_numpys(embed3, embed4)}")

Similarity (robot to robot): 0.9156871438026428
Similarity (robot to me   ): 0.8400258421897888
Similarity (me 1 to me 2  ): 0.8794167637825012


/tmp/ipykernel_3047664/3060022475.py:14: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  dot = np.dot(v1, v2)


## store

In [40]:
class NHWCWrapper(nn.Module):
    def __init__(self, model_nchw):
        super().__init__()
        self.model = model_nchw

    def forward(self, x):
        # x: NHWC => NCHW
        x = x.permute(0, 3, 1, 2).contiguous()
        return self.model(x)

In [41]:

import onnx

def export_to_onnx(model, onnx_path="ReDimNet_no_mel.onnx"):
    model.eval()
    
    # Create a dummy input with shape matching [B=1, 1, n_mels=72, time_frames=200] (example)
    dummy_input = torch.randn(1, 1, 60, 200)
    
    model_NHWC = NHWCWrapper(model)
    dummy_input_NHWC = torch.randn(1, 60, 200, 1)
    
    #  fixed-length segments 
    torch.onnx.export(
        model_NHWC,
        dummy_input_NHWC,
        onnx_path,
        input_names=["log_mel"],
        output_names=["embedding"],
        opset_version=11
    )
    
    # # dynamic axes for variable time frames
    # torch.onnx.export(
    #     model_NHWC,
    #     dummy_input_NHWC,
    #     onnx_path,
    #     input_names   = ["log_mel"],
    #     output_names  = ["embedding"],
    #     opset_version = 11,               # use a recent opset
    #     dynamic_axes = {
    #         # input  tensor : {axis_index : symbolic_name}
    #         "log_mel"  : {0: "batch",   2: "time"},   # B and T now flexible
    #         "embedding": {0: "batch"}                 # output length is fixed, batched
    #     }
    # )   
    
    print("Exported to", onnx_path)

# Example usage
export_to_onnx(model_no_mel)

Backbone output shape: torch.Size([1, 600, 200])
Exported to ReDimNet_no_mel.onnx


In [42]:
!ls -lah ReDimNet_no_mel.onnx

-rw-rw-r-- 1 vlad vlad 4.0M Jun 17 10:16 ReDimNet_no_mel.onnx


## verify

In [43]:
import onnx
onnx_model = onnx.load("ReDimNet_no_mel.onnx")
onnx.checker.check_model(onnx_model)
print("ONNX model is valid!")


ONNX model is valid!


In [44]:
print(type(model_no_mel))
print(isinstance(model_no_mel, nn.Module))

<class '__main__.ReDimNetNoMel'>
True


In [45]:
type(onnx_model)

onnx.onnx_ml_pb2.ModelProto

In [46]:
import onnxruntime as ort

def run_inference_onnx(onnx_path, wav_path):
    """
    Loads an audio file, converts to log-mel, and runs inference
    in an ONNX session. Returns the embedding as a NumPy array.
    """
    #######################################
    # 1) Load your ONNX model
    #######################################
    # (Optional) onnx.checker to confirm it’s valid
    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    print(f"Loaded and checked ONNX model from: {onnx_path}")

    # Create an inference session
    session = ort.InferenceSession(onnx_path)

    # Usually we retrieve the first input & output name
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    #######################################
    # 2) Load audio, get log-mel
    #######################################
    waveform, sample_rate = torchaudio.load(wav_path)
    # If multi-channel, downmix:
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
        
    # Resample if needed
    target_sample_rate=16000
    if sample_rate != target_sample_rate:
        resampler = T.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = resampler(waveform)

    log_mel = waveform_to_logmel(waveform)

    #######################################
    # 3) ONNX Inference
    #######################################
    # Convert to NumPy for ONNX runtime
    log_mel_np = log_mel.cpu().numpy()
    log_mel_np = np.transpose(log_mel_np, (0, 2, 3, 1))

    # Run inference
    outputs = session.run([output_name], {input_name: log_mel_np})
    # outputs is a list; typically we want the first item
    embedding = outputs[0]  # shape is [1, embedding_dim]

    print("Embedding shape:", embedding.shape)
    # print("Embedding data:\n", embedding)
    return embedding

In [47]:
onnx_model_path = "ReDimNet_no_mel.onnx"

embed1 = run_inference_onnx(onnx_model_path, "testRob1.wav")
embed2 = run_inference_onnx(onnx_model_path, "testRob2.wav")
embed3 = run_inference_onnx(onnx_model_path, "testme1.wav")
embed4 = run_inference_onnx(onnx_model_path, "testme2.wav")






Loaded and checked ONNX model from: ReDimNet_no_mel.onnx
Padding log_mel from 108 to 200 frames
Embedding shape: (1, 192)
Loaded and checked ONNX model from: ReDimNet_no_mel.onnx
Padding log_mel from 99 to 200 frames
Embedding shape: (1, 192)
Loaded and checked ONNX model from: ReDimNet_no_mel.onnx
Padding log_mel from 118 to 200 frames
Embedding shape: (1, 192)
Loaded and checked ONNX model from: ReDimNet_no_mel.onnx
Cropping log_mel from 222 to 200 frames
Embedding shape: (1, 192)


In [48]:
print(f"Similarity (robot to robot): {cosine_similarity_numpys(embed1, embed2)}")
print(f"Similarity (robot to me   ): {cosine_similarity_numpys(embed1, embed3)}")
print(f"Similarity (me 1 to me 2  ): {cosine_similarity_numpys(embed3, embed4)}")

Similarity (robot to robot): 0.9156870245933533
Similarity (robot to me   ): 0.8400259613990784
Similarity (me 1 to me 2  ): 0.8794167637825012


## cal fake data

* run in rknn docker:

``` bash

import os
import numpy as np
import torch

# Directory for calibration inputs
os.makedirs("calib_npy", exist_ok=True)

# Create 100 dummy log-mel tensors
for i in range(10):
    log_mel = torch.randn(1, 1, 60, 200).numpy().astype(np.float32)
    np.save(f"calib_npy/sample_{i}.npy", log_mel)

# Write dataset.txt listing all paths
with open("dataset.txt", "w") as f:
    for i in range(10):
        f.write(f"calib_npy/sample_{i}.npy\n")


````

## converts

* python convert.py ReDimNet_no_mel.onnx  rk3588 fp ReDimNet_no_mel.rknn 
* python convert.py ReDimNet_no_mel.onnx  rv1106 i8  ReDimNet_no_mel.rknn